# Chat 💬 (Bronze &rarr; Silver)

## Init

Import dependencies

In [ ]:
import findspark
from decouple import Config, RepositoryEnv
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.corpus import stopwords
import pandas as pd

In [ ]:
nltk.download('all')

findspark.init()
findspark.add_packages('mysql:mysql-connector-java:8.0.11')

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.ml.feature import StopWordsRemover

from pyspark.sql.functions import pandas_udf 
from pyspark import Row 

Get *.env* values

In [ ]:
# Env values
# ENV_PATH = os.path.join(os.path.dirname(__file__), '../.env')
env = Config(RepositoryEnv('../.env'))
db_server = env.get('DB_IP')
db_name = env.get('DB_DATABASE')
db_user = env.get('DB_USERNAME')
db_password = env.get('DB_PASSWORD')

## Start

Create Spark session

In [ ]:
spark = SparkSession.builder \
    .appName("PySpark SQL Server Connection") \
    .getOrCreate()

# .config("spark.jars", "file:///C:/spark-3.4.1-bin-hadoop3/jars/mssql-jdbc-12.6.1.jre8.jar") \

Start database connection and data **extraction** from bronze table

In [ ]:
jdbc_url = f"jdbc:sqlserver://{db_server};databaseName={db_name};user={db_user};password={db_password};encrypt=true;trustServerCertificate=true;"
db_table = "Twitch.MessagesStg"

employees_df = spark.read \
    .format("jdbc") \
    .option("url", jdbc_url) \
    .option("dbtable", db_table) \
    .load()

employees_df.show(truncate=False)

**Transform** data

In [ ]:
SPLIT_SEPARATOR = "SPLIT-SEPARATOR"
# Remove all line breaks
e1_df = employees_df.withColumn('MsgResponse',regexp_replace(col('MsgResponse'),'\r\n',' '))
# Add a key word at the begining of each command to then know where do we need to split
e1_df = e1_df.withColumn("MsgResponse", regexp_replace(col('MsgResponse'), '(:[^:!]+![^!@]+@[^@.]+\.tmi\.twitch\.tv PRIVMSG)(?!$)',  f"{SPLIT_SEPARATOR}"+r"$1"))
e1_df = e1_df.withColumn("MessageResponseSplit", split(col('MsgResponse'),SPLIT_SEPARATOR))
# Remove first element of split list (empty element)
e1_df = e1_df.withColumn("MessageResponseSplit", expr("slice(MessageResponseSplit, 2, SIZE(MessageResponseSplit))"))
# Create a row for each element
e1_df = e1_df.withColumn("MsgResponse", explode(col('MessageResponseSplit')))

# Split to get each section of the command
splitted_col = split(e1_df['MsgResponse'],' ')

e1_df = e1_df.withColumn('Username', regexp_extract(splitted_col.getItem(0), ':(.*)!', 1))
e1_df = e1_df.withColumn('Command', splitted_col.getItem(1))
e1_df = e1_df.withColumn('Channel', regexp_replace(splitted_col.getItem(2),'#',''))
e1_df = e1_df.withColumn('MsgResponseMeta', concat_ws(' ',splitted_col.getItem(0),splitted_col.getItem(1),splitted_col.getItem(2),lit(':'))) # Aux
e1_df = e1_df.withColumn('Message', regexp_replace(col('MsgResponse'),col('MsgResponseMeta'),' '))

# e1_df = e1_df.withColumn('Tag', regexp_extract(col('Message'), '@(.*) ', 1))

# Drop columns that are not needed
e1_df = e1_df.drop('Id','MsgResponse','MessageResponseSplit','MsgResponseMeta')

e1_df.show(truncate=False)

In [ ]:
# Keep only the command that are chat related ('PRIVMSG') and generate an id for each record
e2_df = e1_df \
    .where(col('Command')=='PRIVMSG') \
    .withColumn("Id", monotonically_increasing_id()) \
    .select(col('Id'), col('Date'), col('Username'), col('Command'), col('Channel'), col('Message'))

e2_df.show(truncate=False)

In [ ]:
# UDF for sentiment analysis
analyzer = SentimentIntensityAnalyzer()
@pandas_udf('float')  
def sentiment_analysis(s: pd.Series) -> pd.Series:
  return s.apply(lambda msg: analyzer.polarity_scores(msg)['compound'])

# Tokenize words
e3_df = e2_df.withColumn("MessageClean1", split(lower(col("Message")),' '))

# Remove Stop Words
sw_remover = StopWordsRemover(stopWords=stopwords.words('spanish') + stopwords.words('english'))
sw_remover.setInputCols(["MessageClean1"]).setOutputCols(["MessageClean2"])

e3_df = sw_remover.transform(e3_df).withColumn('MessageClean2', concat_ws(' ', col('MessageClean2')))

# Get sentiment analysis score with VADER dict
e3_df = e3_df.withColumn('SentimentScore',sentiment_analysis(col('MessageClean2')))

# Drop columns that are not needed
e3_df = e3_df.drop('MessageClean1','MessageClean2')

e3_df.show(truncate=False)

In [ ]:
# Need to fix jars
# from sparknlp.pretrained import PretrainedPipeline 
# pipeline = PretrainedPipeline("translate_es_en", lang = "xx") 
# pipeline.annotate("Your sentence to translate!")

**Load** data into database silver table

In [ ]:
output_table = 'Twitch.MessagesRef'

e3_df.write \
  .format("jdbc") \
  .mode("overwrite") \
  .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
  .option("url", jdbc_url) \
  .option("dbtable", output_table) \
  .save() 

Stop Spark session

In [ ]:
spark.stop()